In [322]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select

import numpy as np
import pandas as pd

options = Options()

#Comment this out if you want to see the browser
#options.add_argument("--headless")

pd.set_option('display.max_rows', 1000)

driver = webdriver.Chrome(options=options)

In [323]:
def get_prop_lines(select, value):
    #Selects the prop passed as 'value' here from dropdown menu
    select.select_by_value(value)
    driver.implicitly_wait(10)

    #Initializes dataframe to store results
    result = pd.DataFrame(columns=["name", "team", "lines", "odds"])

    #fetches all games for a given prop and store them in 'games'
    games_cont = driver.find_element(By.CLASS_NAME, "wrapper")
    games = games_cont.find_elements(By.TAG_NAME, "app-game")

    for game in games:
        # Running into persistent proglem where the buttons for the individual games are not clicked properly so 
        # user has to click them when the program gets there (need to turn off headless mode to see this)
        # 4 lines below works for the MX games but not MLS for some reason
        driver.implicitly_wait(15)
        game.click()
        driver.implicitly_wait(2)
        game.click()

        game_content = WebDriverWait(game, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".content")))
        #Stores each individual prop bet in 'players'
        players = game_content.find_elements(By.CLASS_NAME, "player-group")
        print(f"Players: {len(players)}")
        for player in players:
            #Gets attributes for each individual prop bet and puts them all into one dataframe
            name = player.find_element(By.TAG_NAME, "b").text
            team = player.find_element(By.CLASS_NAME, "ss-team-names").text
            lines = player.find_element(By.TAG_NAME, "option").get_attribute("value")
            line = lines[0]
            odds = player.find_element(By.TAG_NAME, "button").text

            wager = pd.DataFrame([{"name": name, "team": team, "lines": line, "odds": odds}])
            frames = [result, wager]
            result = pd.concat(frames, ignore_index=True)

            print(name, team, line, odds)

        game.click()

    return result

In [324]:
#Source function to run the scraper
def bovScrape_league_props(url):
    driver.get(url)
    driver.implicitly_wait(5)

    #Navigates to part of page where the prop bets are, enters the iframe and accesses the dropdown menu
    wait = WebDriverWait(driver, 10)
    league_box = wait.until(EC.presence_of_element_located((By.ID, "playerPropsLeagueFrame")))

    league_frame = league_box.find_element(By.TAG_NAME, "iframe")
    driver.switch_to.frame(league_frame)

    dropdown = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "select")))
    select = Select(dropdown)

    shots_lines = get_prop_lines(select, "Shots")
    shots_on_goal_lines = get_prop_lines(select, "Shots on Goal")


    return shots_lines, shots_on_goal_lines
    

In [325]:
mex_url = 'https://www.bovada.lv/sports/soccer/north-america/mexico/liga-mx-apertura'
mls_url = 'https://www.bovada.lv/sports/soccer/north-america/united-states/mls'
shots, sogs = bovScrape_league_props(mls_url)
shots

Players: 29
Alexandru Matan CLB 2 +197
Christian Ramirez CLB 2 -173
Cucho Hernandez CLB 3 -139
Darlington Nagbe CLB 2 +440
Gustavo Vallecilla CLB 2 +925
Jacen Russell-Rowe CLB 2 -159
Kevin Molino CLB 2 +146
Lucas Zelarrayan CLB 2 -150
Luis Diaz CLB 2 +275
Milos Degenek CLB 2 +865
Mohamed Ramzdine Farsi CLB 2 +865
Philip Quinton CLB 2 +865
Sean Zawadzki CLB 2 +530
Steven Moreira CLB 2 +955
Yaw Yeboah CLB 2 +150
Benjamin Cremaschi MIA 2 +167
Christopher McVey MIA 2 +955
David Ruiz MIA 2 +580
DeAndre Yedlin MIA 2 +990
Dixon Arroyo MIA 2 +420
Josef Martinez MIA 2 +104
Kamal Miller MIA 2 +894
Leonardo Campana MIA 2 +106
Nicolas Stefanelli MIA 2 +150
Robbie Robinson MIA 2 -145
Robert Taylor MIA 2 +270
Rodolfo Pizarro MIA 2 +242
Ryan Sailor MIA 2 +990
Sergey Krivtsov MIA 2 +825
Players: 35
Antonio Carlos OCS 2 +484
Cesar Araujo OCS 2 +515
Duncan McGuire OCS 2 +138
Ercan Kara OCS 2 +131
Facundo Torres OCS 2 -122
Felipe Martins OCS 2 +295
Gaston Gonzalez OCS 2 -103
Ivan Angulo OCS 2 +104
Kyle S

WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00A4A813+48355]
	(No symbol) [0x009DC4B1]
	(No symbol) [0x008E5358]
	(No symbol) [0x008D83E5]
	(No symbol) [0x008D82EC]
	(No symbol) [0x008D6F26]
	(No symbol) [0x008D77AA]
	(No symbol) [0x008E8417]
	(No symbol) [0x008E99E1]
	(No symbol) [0x008E9A1E]
	(No symbol) [0x008E9A80]
	(No symbol) [0x0090D2FD]
	(No symbol) [0x0092A73C]
	(No symbol) [0x00909A36]
	(No symbol) [0x0092AA94]
	(No symbol) [0x0093C922]
	(No symbol) [0x0092A536]
	(No symbol) [0x009082DC]
	(No symbol) [0x009093DD]
	GetHandleVerifier [0x00CAAABD+2539405]
	GetHandleVerifier [0x00CEA78F+2800735]
	GetHandleVerifier [0x00CE456C+2775612]
	GetHandleVerifier [0x00AD51E0+616112]
	(No symbol) [0x009E5F8C]
	(No symbol) [0x009E2328]
	(No symbol) [0x009E240B]
	(No symbol) [0x009D4FF7]
	Ba

In [ ]:
sogs

,name,team,lines,odds
0,Alan Mozo,GDC,1,+300
1,Alan Torres,GDC,1,+400
2,Alejandro Mayorga,GDC,1,+400
3,Alexis Vega,GDC,1,-223
4,Antonio Briseno,GDC,1,+400
5,Carlos Cisneros,GDC,1,+100
6,Cristian Calderon,GDC,1,+200
7,Daniel Rios,GDC,1,+100
8,Fernando Beltran,GDC,1,+200
9,Gilberto Sepulveda,GDC,1,+500
